In [1]:
import numpy as np
import struct
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
import torchvision
from copy import deepcopy
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import time
from scipy import spatial
from util.util import mnist_noise

from trajectoryReweight.model import WeightedCrossEntropyLoss, TrajectoryReweightNN
from trajectoryReweight.baseline import StandardTrainingNN
from trajectoryReweight.gmm import GaussianMixture

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

cuda:0


In [2]:
"""
CNN
"""
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
    
def accuracy(predict_y, test_y):
    score = 0
    for pred, acc in zip(predict_y, test_y):
        if pred == acc:
            score +=1
    return score / test_y.shape[0]

In [3]:
"""
MNIST DATA
"""
x_train = read_idx('data/train-images.idx3-ubyte')
y_train = read_idx('data/train-labels.idx1-ubyte')
x_test = read_idx('data/t10k-images.idx3-ubyte')
y_test = read_idx('data/t10k-labels.idx1-ubyte')
valid_idx = np.random.choice(range(60000), size=1000, replace=False)
x_valid = x_train[valid_idx]
y_valid = y_train[valid_idx]
x_train = np.delete(x_train, valid_idx, axis=0)
y_train = np.delete(y_train, valid_idx)

"""
shrink dataset to make noisy significant
"""
subset_idx = np.random.choice(range(60000-1000), size=10000, replace=False)
x_train = x_train[subset_idx]
y_train = y_train[subset_idx]

"""
Add Noise label to training data
"""
y_train_noisy, noise_index = mnist_noise(y_train,0.1)

In [4]:
"""
Initializaion
"""
train_idx = np.arange(len(x_train))
x_train = np.transpose(x_train,(2,1,0))
x_valid = np.transpose(x_valid,(2,1,0))
x_test = np.transpose(x_test,(2,1,0))
x_train_tensor = torchvision.transforms.ToTensor()(x_train).unsqueeze(1)
x_valid_tensor = torchvision.transforms.ToTensor()(x_valid).unsqueeze(1)
x_test_tensor = torchvision.transforms.ToTensor()(x_test).unsqueeze(1)
y_train_tensor = torch.from_numpy(y_train.astype(np.long))
y_train_noisy_tensor = torch.from_numpy(y_train_noisy.astype(np.long))
y_valid_tensor = torch.from_numpy(y_valid.astype(np.long))
y_test_tensor = torch.from_numpy(y_test.astype(np.long))

In [5]:
"""
CNN without reweight
"""
cnn = CNN()
cnn.to(device)

stand_trainNN = StandardTrainingNN(cnn,
                                   batch_size=64,
                                   num_iter=80,
                                   learning_rate=1e-3,
                                   early_stopping=7,
                                   device=device,
                                   iprint=1)
stand_trainNN.fit(x_train_tensor, y_train_noisy_tensor, x_valid_tensor, y_valid_tensor, x_test_tensor, y_test_tensor)

test_output_y = stand_trainNN.predict(x_test_tensor)
test_accuracy = accuracy(test_output_y, y_test_tensor.data.numpy())

print('test accuracy is {}%'.format(100 * test_accuracy))

Standard NN training...
epoch = 1 | training loss = 0.9581 | valid loss = 0.0049 | valid accuarcy = 96.5% | early stopping = 0/7 | test loss = 0.0046 | test accuarcy = 96.25% [9625/10000]
epoch = 2 | training loss = 0.7433 | valid loss = 0.0048 | valid accuarcy = 97.3% | early stopping = 0/7 | test loss = 0.0044 | test accuarcy = 97.19% [9719/10000]
epoch = 3 | training loss = 0.7087 | valid loss = 0.0049 | valid accuarcy = 96.6% | early stopping = 1/7 | test loss = 0.0045 | test accuarcy = 97.4% [9740/10000]
epoch = 4 | training loss = 0.6796 | valid loss = 0.0036 | valid accuarcy = 96.5% | early stopping = 0/7 | test loss = 0.0030 | test accuarcy = 97.11% [9711/10000]
epoch = 5 | training loss = 0.6595 | valid loss = 0.0028 | valid accuarcy = 98.5% | early stopping = 0/7 | test loss = 0.0025 | test accuarcy = 98.13% [9813/10000]
epoch = 6 | training loss = 0.6398 | valid loss = 0.0028 | valid accuarcy = 98.7% | early stopping = 1/7 | test loss = 0.0026 | test accuarcy = 98.3% [9830/1

In [ ]:
"""
CNN with reweight
"""
cnn = CNN()
cnn.to(device)

tra_weightNN = TrajectoryReweightNN(cnn,
                                    burnin=3,
                                    num_cluster=10,
                                    batch_size=64,
                                    num_iter=80,
                                    learning_rate=1e-3,
                                    early_stopping=7,
                                    device=device,
                                    traj_step=1,
                                    iprint=2)
tra_weightNN.fit(x_train_tensor, y_train_noisy_tensor, x_valid_tensor, y_valid_tensor,x_test_tensor, y_test_tensor, noise_index)

test_output_y = tra_weightNN.predict(x_test_tensor)
test_accuracy = accuracy(test_output_y, y_test_tensor.data.numpy())

print('test accuracy is {}%'.format(100 * test_accuracy))

Train 3 burn-in epoch...
epoch = 1 | test loss = 0.0059 | test accuarcy = 92.05% [9205/10000]
epoch = 2 | test loss = 0.0058 | test accuarcy = 95.57% [9557/10000]
epoch = 3 | test loss = 0.0051 | test accuarcy = 95.58% [9558/10000]
Train 3 burn-in epoch complete.
------------------------------------------------------------
Trajectory clustering for burn-in epoch...
| - {0: 0, 'size': 925, 'sim': '0.9868', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {1: 1, 'size': 1756, 'sim': '0.9940', 'num_special': 1, 'spe_ratio': '0.0006'}
| - {2: 2, 'size': 903, 'sim': '0.9917', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {3: 3, 'size': 1118, 'sim': '0.9893', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {4: 4, 'size': 1170, 'sim': '0.9941', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {5: 5, 'size': 636, 'sim': '0.9929', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {6: 6, 'size': 519, 'sim': '0.9916', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {7: 7, 'size': 1018, 'sim': '0.3957', 'num_special'

| - {9: 9, 'size': 1003, 'sim': '-0.7665', 'num_special': 1, 'spe_ratio': '0.0010'}
epoch = 11 | training loss = 0.5957 | valid loss = 0.0029 | valid accuarcy = 98.0% | early stopping = 1/7 | test loss = 0.0026 | test accuarcy = 98.13% [9813/10000]
| - {0: 0, 'size': 1137, 'sim': '0.9827', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {1: 1, 'size': 358, 'sim': '-0.8404', 'num_special': 347, 'spe_ratio': '0.9693'}
| - {2: 2, 'size': 388, 'sim': '-0.7499', 'num_special': 48, 'spe_ratio': '0.1237'}
| - {3: 3, 'size': 1210, 'sim': '-0.6624', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {4: 4, 'size': 609, 'sim': '-0.8450', 'num_special': 604, 'spe_ratio': '0.9918'}
| - {5: 5, 'size': 1724, 'sim': '-0.8273', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {6: 6, 'size': 1496, 'sim': '-0.8256', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {7: 7, 'size': 892, 'sim': '-0.8154', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {8: 8, 'size': 985, 'sim': '-0.7707', 'num_special': 1, 'spe_ratio': '0.001

| - {0: 0, 'size': 353, 'sim': '0.9015', 'num_special': 46, 'spe_ratio': '0.1303'}
| - {1: 1, 'size': 1783, 'sim': '0.9587', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {2: 2, 'size': 808, 'sim': '0.9623', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {3: 3, 'size': 899, 'sim': '0.9775', 'num_special': 1, 'spe_ratio': '0.0011'}
| - {4: 4, 'size': 331, 'sim': '-0.8048', 'num_special': 322, 'spe_ratio': '0.9728'}
| - {5: 5, 'size': 1644, 'sim': '-0.7732', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {6: 6, 'size': 1421, 'sim': '-0.6845', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {7: 7, 'size': 1020, 'sim': '-0.6678', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {8: 8, 'size': 1110, 'sim': '-0.6057', 'num_special': 0, 'spe_ratio': '0.0000'}
| - {9: 9, 'size': 631, 'sim': '-0.9193', 'num_special': 631, 'spe_ratio': '1.0000'}
epoch = 20 | training loss = 0.3304 | valid loss = 0.0018 | valid accuarcy = 98.3% | early stopping = 0/7 | test loss = 0.0015 | test accuarcy = 98.68% [9868/10000]

In [ ]:
"""
Different noise level
"""

noise_level = [0.1,0.2,0.4,0.6,0.8]
exp_summary = []

for level in noise_level:
    
    level_summary = {"noise_level": level, "std_test":0, "rewgt_test":0}       
        
    np.random.seed(int(time.time()))
    x_train = read_idx('data/train-images.idx3-ubyte')
    y_train = read_idx('data/train-labels.idx1-ubyte')
    x_test = read_idx('data/t10k-images.idx3-ubyte')
    y_test = read_idx('data/t10k-labels.idx1-ubyte')
    valid_idx = np.random.choice(range(60000), size=1000, replace=False)
    x_valid = x_train[valid_idx]
    y_valid = y_train[valid_idx]
    x_train = np.delete(x_train, valid_idx, axis=0)
    y_train = np.delete(y_train, valid_idx)

    y_train_noisy = mnist_noise(y_train, level) #apply noise level
 
    train_idx = np.arange(len(x_train))
    x_train = np.transpose(x_train,(2,1,0))
    x_valid = np.transpose(x_valid,(2,1,0))
    x_test = np.transpose(x_test,(2,1,0))
    x_train_tensor = torchvision.transforms.ToTensor()(x_train).unsqueeze(1)
    x_valid_tensor = torchvision.transforms.ToTensor()(x_valid).unsqueeze(1)
    x_test_tensor = torchvision.transforms.ToTensor()(x_test).unsqueeze(1)
    y_train_tensor = torch.from_numpy(y_train.astype(np.long))
    y_train_noisy_tensor = torch.from_numpy(y_train_noisy.astype(np.long))
    y_valid_tensor = torch.from_numpy(y_valid.astype(np.long))
    y_test_tensor = torch.from_numpy(y_test.astype(np.long))

    cnn = CNN()
    cnn.to(device)

    stand_trainNN = StandardTrainingNN(cnn,
                                       batch_size=64,
                                       num_iter=80,
                                       learning_rate=1e-3,
                                       early_stopping=5,
                                       device=device,
                                       iprint=0)
    stand_trainNN.fit(x_train_tensor, y_train_noisy_tensor, x_valid_tensor, y_valid_tensor, x_test_tensor, y_test_tensor)

    test_output_y = stand_trainNN.predict(x_test_tensor)
    test_accuracy = accuracy(test_output_y, y_test_tensor.data.numpy())
    level_summary['std_test'] = test_accuracy

    time.sleep(10)

    cnn = CNN()
    cnn.to(device)

    tra_weightNN = TrajectoryReweightNN(cnn,
                                        burnin=5,
                                        num_cluster=5,
                                        batch_size=64,
                                        num_iter=80,
                                        learning_rate=1e-3,
                                        early_stopping=5,
                                        device=device,
                                        traj_step = 3,
                                        iprint=0)
    tra_weightNN.fit(x_train_tensor, y_train_noisy_tensor, x_valid_tensor, y_valid_tensor,x_test_tensor, y_test_tensor)

    test_output_y = tra_weightNN.predict(x_test_tensor)
    test_accuracy = accuracy(test_output_y, y_test_tensor.data.numpy())
    level_summary['rewgt_test'] = test_accuracy

    time.sleep(10)
    
    exp_summary.append(level_summary)
    print('nosie level = {} finished running'.format(level))
    time.sleep(30)

In [ ]:
noise_index == False

In [ ]:
print(exp_summary)